In [1]:
import os
from bs4 import BeautifulSoup
from natasha import NamesExtractor  
import json
extractor = NamesExtractor()
import re
#регулярка для имён
fio = re.compile("[А-ЯЁ][а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.")
fio_cifer = re.compile("[А-ЯЁ][а-яё]+ ФИО[0-9]{1,3}")
fio_short = re.compile("ФИО[0-9]{1,3}")
import pandas as pd
import random

In [2]:
from meta_extraction import final_meta, meta_lines, get_meta_names
from meta_extraction import get_parts, parse_xml

In [3]:
import numpy as np

In [4]:
def getting_paths(path_to_folder):
    paths = []
    for f in os.walk(path_to_folder):
        for path in f[2]:
            paths.append(path_to_folder+"/"+path)
    return paths

In [5]:
def only_criminal(paths):
    new_paths = []
    for path in paths:
        text = ""
        with open(path, "r") as f:
            for line in f.readlines():
                text += line
        if "<vidpr>Уголовное</vidpr>" not in text: 
            os.remove(path)
        else:
            new_paths.append(path) 
    return new_paths            

In [6]:
persons_meta = ["prosecutor", "advocate", "accused", "secretary"]

In [8]:
def random_corpus(link, number):
    links_to_folders = []
    links_to_files = []
    for x in os.walk(link):
        links_to_folders.append(x[0])
    for link in links_to_folders:
        links_to_files += getting_paths(link)
    all_links_to_files = [link for link in links_to_files if "//" not in link]
    corpus_link = random.sample(all_links_to_files, number)
    return corpus_link

In [9]:
rand_links = random_corpus("/home/laidhimonthegreen/Документы/coursework/evaluation_data/", 100) 

In [10]:
heads = []
for link in rand_links:
    heads.append(get_parts(parse_xml(link)[1])[1])

In [148]:
with open("herads.txt", "w") as f:
    for head in heads:
        f.write(head + "\n")

In [5]:
goldstandard = pd.DataFrame.from_csv("/home/laidhimonthegreen/Документы/coursework/gold_standard.csv", sep = "\t")

/home/laidhimonthegreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [6]:
roles = ["number","prosecutor", "accused", "secretary", "advocate"]

In [7]:
script_martkup_df = pd.DataFrame(columns = roles)

In [8]:
prosecutors = []
accuseds = []
secretaries = []
advocates = []

In [9]:
our_links = []
with open("links_standard.txt", "r") as f:
    for line in f.readlines():
        our_links.append(line.strip())

In [10]:
get_meta_names(meta_lines(our_links[39]))

[('prosecutor', 'Будаев В.В.'),
 ('accused', 'Фелифаров Ф.А.'),
 ('advocate', 'Цыреновой Н.Р.'),
 ('secretary', 'Хантаева Н.В.'),
 ('accused', 'Фелифаров Ф.А.')]

In [11]:
for i, link in enumerate(our_links): 
    try: 
        roledict = final_meta(link)    
        accuseds.append(roledict["accused"])
        prosecutors.append(roledict["prosecutor"])
        secretaries.append(roledict["secretary"])
        advocates.append(roledict["advocate"])
    except IndexError:
        prosecutors.append("not found")
        accuseds.append("not found")
        secretaries.append("not found")
        advocates.append("not found")

In [12]:
secretaries[0:10]

['Щербинина С.Ф.',
 'Тугульдурова Э.Ц.',
 'Тихановой А.Ю.',
 'Самойленко Е.В.',
 'Куськеева Л.И.',
 'Куськеева Л.И.',
 'Золоева Э.Г.',
 'not found',
 'Манджиева О.Д.',
 'Сухарева Е.П.']

In [13]:
goldstandard.head()

,prosecutor,accused,secretary,advocate
number,,,,
1,Амантаев Р.Р.,Рахимов ФИО13,Щербинина С.Ф.,Гиниятуллин Ф.Н.
2,Черкасова О.Ю.,Тимофеев А.В.,Тугульдурова Э.Ц.,Иванова О.В.
3,Рябцев А.И.,КВВ,Тиханова А.Ю.,Зуев В.И.
4,Рябцев А.И.,Астафьева Н.Е.,Самойленко Е.В.,Салмина Ю.В.
5,Абушинов В.В.,Маджаров В.Б.,Куськеева Л.И.,Павлов Р.Б.


In [32]:
def names_comparison(name1, name2):
    a = len(name1.split()[0][0:-2])
    name1_splitted = name1.split()[0][:a] + " " + name1.split()[1]
    name2_splitted = name2.split()[0][:a] + " " + name2.split()[1] 
    if name1_splitted == name2_splitted: 
        equal = 1
    else:
        equal = 0
    return equal

In [33]:
def wrong_or_right(gold_words, predicted_words, cases):
    count_right = 0
    count_wrong = 0
    for i, word in enumerate(gold_words):
        if cases == False:
            if word == predicted_words[i]:
                count_right += 1
            else:
                count_wrong += 1 
        if cases == True:
            try:
                if names_comparison(word, predicted_words[i]) == 1:
                    count_right += 1
                else:
                    count_wrong += 1  
            except IndexError:
                if word == predicted_words[i]:
                    count_right += 1
                else:
                    count_wrong += 1   
    return (count_right/len(gold_words), count_wrong/len(gold_words))

In [34]:
secretaries_gold = list(goldstandard["secretary"])
accuseds_gold = list(goldstandard["accused"])
prosecutors_gold = list(goldstandard["prosecutor"])
advocates_gold = list(goldstandard["advocate"])

In [35]:
wrong_or_right(secretaries_gold, secretaries, False)

(0.6, 0.4)

In [36]:
wrong_or_right(secretaries_gold, secretaries, True)

(0.83, 0.17)

In [37]:
wrong_or_right(accuseds_gold, accuseds, True)

(0.61, 0.39)

In [38]:
wrong_or_right(prosecutors_gold, prosecutors, True)

(0.85, 0.15)

In [39]:
wrong_or_right(all_people_gold, all_people, True)

(0.7825, 0.2175)

In [40]:
wrong_or_right(advocates_gold, advocates, True)

(0.84, 0.16)

In [41]:
all_people = advocates + secretaries + accuseds + prosecutors
all_people_gold = advocates_gold + secretaries_gold + accuseds_gold + prosecutors_gold